In [ ]:
# export
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from transformers import AutoModel
from fastcore.all import *

from deep_t2i.torch_utils import *

In [ ]:
# default_exp model

# Model
> 

## DAMSM and Encoder

In [ ]:
# export
class RnnEncoder(nn.Module):
    def __init__(self, vocab_sz, emb_sz, pad_id, n_layers=2, drop_p=0.5):
        super().__init__()
        self.n_layers = n_layers
        self.emb_sz = emb_sz
        self.pad_id = pad_id
        self.emb = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_id)
        self.emb.weight.data.uniform_(-0.1, 0.1)
        self.gru = nn.GRU(emb_sz, emb_sz//2, num_layers=n_layers, batch_first=True, dropout=drop_p, bidirectional=True)
    def forward(self, inp_ids, true_seqlen):
        ''' inp_ids: (bs, seq_len), true_seqlen: (bs,)
            returns: (bs, emb_sz), (bs, seq_len, emb_sz) '''
        bs, seq_len = inp_ids.shape
        emb = self.emb(inp_ids) # (bs, seq_len, emb_sz)
        emb_packed = pack_padded_sequence(emb, true_seqlen, batch_first=True, enforce_sorted=False)
        
        word_emb_packed, hn = self.gru(emb_packed) # hn: (n_layers*2, bs, emb_sz//2)
        word_emb, _ = pad_packed_sequence(word_emb_packed, batch_first=True, total_length=seq_len) # (bs, seq_len, emb_sz)

        hn = hn.view(self.n_layers, 2, bs, self.emb_sz//2) # (self.n_layers, 2, bs, self.emb_sz//2)
        sent_emb = torch.cat([hn[-1][0], hn[-1][1]], 1)
        return sent_emb, word_emb

In [ ]:
encoder = RnnEncoder(vocab_sz=111, emb_sz=400, pad_id=1)
inp_ids = torch.randint(0, 100, (16, 20))
true_seqlen = torch.randint(1, 20, (16,))
sent_emb, word_emb = encoder(inp_ids, true_seqlen)
test_eq(sent_emb.shape, (16, 400))
test_eq(word_emb.shape, (16, 20, 400))

In [ ]:
# export
class CnnEncoder(nn.Module):
    def __init__(self, emb_sz):
        ''' from https://github.com/taoxugit/AttnGAN/blob/master/code/model.py '''
        super().__init__()
        self.emb_sz = emb_sz
        model = torch.hub.load('pytorch/vision:v0.6.0', 'inception_v3', pretrained=True, verbose=False).requires_grad_(False).eval()
        
        self.define_module(model)
        self.init_trainable_weights()

    def define_module(self, model):
        self.Conv2d_1a_3x3 = model.Conv2d_1a_3x3
        self.Conv2d_2a_3x3 = model.Conv2d_2a_3x3
        self.Conv2d_2b_3x3 = model.Conv2d_2b_3x3
        self.Conv2d_3b_1x1 = model.Conv2d_3b_1x1
        self.Conv2d_4a_3x3 = model.Conv2d_4a_3x3
        self.Mixed_5b = model.Mixed_5b
        self.Mixed_5c = model.Mixed_5c
        self.Mixed_5d = model.Mixed_5d
        self.Mixed_6a = model.Mixed_6a
        self.Mixed_6b = model.Mixed_6b
        self.Mixed_6c = model.Mixed_6c
        self.Mixed_6d = model.Mixed_6d
        self.Mixed_6e = model.Mixed_6e
        self.Mixed_7a = model.Mixed_7a
        self.Mixed_7b = model.Mixed_7b
        self.Mixed_7c = model.Mixed_7c

        self.emb_features = nn.Conv2d(768, self.emb_sz, kernel_size=1, stride=1, padding=0, bias=False)
        self.emb_cnn_code = nn.Linear(2048, self.emb_sz)
        
        self.upsample = nn.Upsample(size=(299, 299), mode='bilinear', align_corners=False)

    def init_trainable_weights(self):
        initrange = 0.1
        self.emb_features.weight.data.uniform_(-initrange, initrange)
        self.emb_cnn_code.weight.data.uniform_(-initrange, initrange)

    def forward(self, x):
        ''' x: (bs, 3, 299, 299) 
            returns: features: (bs, emb_sz, 17, 17), cnn_code: (bs, emb_sz)'''
        features = None
        # --> fixed-size input: batch x 3 x 299 x 299
        x = self.upsample(x)
        # 299 x 299 x 3
        x = self.Conv2d_1a_3x3(x)
        # 149 x 149 x 32
        x = self.Conv2d_2a_3x3(x)
        # 147 x 147 x 32
        x = self.Conv2d_2b_3x3(x)
        # 147 x 147 x 64
        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 73 x 73 x 64
        x = self.Conv2d_3b_1x1(x)
        # 73 x 73 x 80
        x = self.Conv2d_4a_3x3(x)
        # 71 x 71 x 192

        x = F.max_pool2d(x, kernel_size=3, stride=2)
        # 35 x 35 x 192
        x = self.Mixed_5b(x)
        # 35 x 35 x 256
        x = self.Mixed_5c(x)
        # 35 x 35 x 288
        x = self.Mixed_5d(x)
        # 35 x 35 x 288

        x = self.Mixed_6a(x)
        # 17 x 17 x 768
        x = self.Mixed_6b(x)
        # 17 x 17 x 768
        x = self.Mixed_6c(x)
        # 17 x 17 x 768
        x = self.Mixed_6d(x)
        # 17 x 17 x 768
        x = self.Mixed_6e(x)
        # 17 x 17 x 768

        # image region features
        features = x
        # 17 x 17 x 768

        x = self.Mixed_7a(x)
        # 8 x 8 x 1280
        x = self.Mixed_7b(x)
        # 8 x 8 x 2048
        x = self.Mixed_7c(x)
        # 8 x 8 x 2048
        x = F.avg_pool2d(x, kernel_size=8)
        # 1 x 1 x 2048
        # x = F.dropout(x, training=self.training)
        # 1 x 1 x 2048
        x = x.view(x.size(0), -1)
        # 2048

        # global image features
        cnn_code = self.emb_cnn_code(x)
        # 512
        if features is not None:
            features = self.emb_features(features)
        return features, cnn_code

In [ ]:
cnn_encoder = CnnEncoder(256)
x = torch.randn(2, 3, 229, 229)
features, cnn_code = cnn_encoder(x)
test_eq(features.shape, (2, 256, 17, 17))
test_eq(cnn_code.shape, (2, 256))

Downloading: "https://github.com/pytorch/vision/archive/v0.6.0.zip" to /root/.cache/torch/hub/v0.6.0.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


In [ ]:
# export
def get_pretrained_DAMSM(path, device='cpu'):
    ''' returns: rnn_encoder, cnn_encoder, gamma1, gamma2, gamma3 '''
    state = torch.load(path, map_location=device)
    vocab_sz = state['vocab_sz']
    emb_sz = state['emb_sz']
    pad_id = state['pad_id']
    rnn_layers = state['rnn_layers']
    rnn_drop_p = state['rnn_drop_p']
    rnn_encoder = RnnEncoder(vocab_sz, emb_sz, pad_id, n_layers=rnn_layers).to(device)
    rnn_encoder.load_state_dict(state['rnn_encoder'])
    
    cnn_encoder = CnnEncoder(emb_sz).to(device)
    cnn_encoder.emb_features.load_state_dict(state['cnn_encoder.emb_features'])
    cnn_encoder.emb_cnn_code.load_state_dict(state['cnn_encoder.emb_cnn_code'])
    
    gamma1 = state['gamma1']
    gamma2 = state['gamma2']
    gamma3 = state['gamma3']
    
    return rnn_encoder, cnn_encoder, vocab_sz, emb_sz, pad_id, rnn_layers, rnn_drop_p, gamma1, gamma2, gamma3

## Core Layers

In [ ]:
# exporti
def conv2d(ni, nf, ks, stride, padding, bias=True):
    " conv2d with kaiming init "
    conv = nn.Conv2d(ni, nf, ks, stride, padding, bias=bias)
    nn.init.kaiming_normal_(conv.weight)
    return conv
def spectral_conv2d(ni, nf, ks, stride, padding, bias=True):
    " conv2d with spectral norm "
    return nn.utils.spectral_norm(conv2d(ni, nf, ks, stride, padding, bias=bias))
class MinibatchStdDev(nn.Module):
    """
    Minibatch standard deviation layer for the discriminator, try to prevent mode collapse
    From https://github.com/akanimax/BMSG-GAN/
    """

    def __init__(self):
        """
        derived class constructor
        """
        super().__init__()

    def forward(self, x, alpha=1e-8):
        """
        forward pass of the layer
        :param x: input activation volume
        :param alpha: small number for numerical stability
        :return: y => x appended with standard deviation constant map
        """
        batch_size, _, height, width = x.shape

        # [B x C x H x W] Subtract mean over batch.
        y = x - x.mean(dim=0, keepdim=True)

        # [1 x C x H x W]  Calc standard deviation over batch
        y = torch.sqrt(y.pow(2.).mean(dim=0, keepdim=False) + alpha)

        # [1]  Take average over feature_maps and pixels.
        y = y.mean().view(1, 1, 1, 1)

        # [B x 1 x H x W]  Replicate over group and pixels.
        y = y.repeat(batch_size, 1, height, width)

        # [B x C x H x W]  Append as new feature_map.
        y = torch.cat([x, y], 1)

        # return the computed values:
        return y

In [ ]:
conv = spectral_conv2d(64, 32, 3, 1, 1)
inp = torch.randn(2, 64, 16, 16)
out = conv(inp)
test_eq(out.shape, (2, 32, 16, 16))

In [ ]:
# exporti
def conv_block_g(ni, nf, ks=3, stride=1, padding=1):
    " spectral_conv2d + batchnorm2d + 0.2 leakyReLU "
    return nn.Sequential(
        spectral_conv2d(ni, nf, ks, stride, padding, bias=False),
        nn.BatchNorm2d(nf),
        nn.LeakyReLU(0.2),
    )
class ConvResBlockG(nn.Module):
    def __init__(self, nf):
        super().__init__()
        self.conv = nn.Sequential(
            conv_block_g(nf, nf),
        )
    def forward(self, x):
        " x: (bs, nf, _, _), returns: (bs, nf, _, _) "
        out = self.conv(x)
        return x + out
def conv_block_d(ni, nf, ks=3, stride=1, padding=1):
    " spectral_conv2d + 0.2 leakyReLU "
    return nn.Sequential(
        spectral_conv2d(ni, nf, ks, stride, padding, bias=True),
        nn.LeakyReLU(0.2),
    )
class ConvResBlockD(nn.Module):
    def __init__(self, nf):
        super().__init__()
        self.conv = nn.Sequential(
            conv_block_d(nf, nf),
        )
    def forward(self, x):
        " x: (bs, nf, _, _), returns: (bs, nf, _, _) "
        out = self.conv(x)
        return x + out

In [ ]:
conv = conv_block_g(64, 32, 3, 1, 1)
inp = torch.randn(2, 64, 16, 16)
out = conv(inp)
test_eq(out.shape, (2, 32, 16, 16))

conv = conv_block_d(64, 32, 3, 1, 1)
inp = torch.randn(2, 64, 16, 16)
out = conv(inp)
test_eq(out.shape, (2, 32, 16, 16))

In [ ]:
# exporti
def up_block():
    ''' (bs, _, _, _) -> (bs, _, _*2, _*2) '''
    return nn.Upsample(scale_factor=2)

In [ ]:
up = up_block()
inp = torch.randn(2, 3, 16, 16)
out = up(inp)
test_eq(out.shape, (2, 3, 32, 32))

In [ ]:
# exporti
def down_block():
    ''' (bs, _, _, _) -> (bs, _, _//2, _//2) '''
    return nn.AvgPool2d(2)

In [ ]:
down = down_block()
inp = torch.randn(2, 3, 32, 32)
out = down(inp)
test_eq(out.shape, (2, 3, 16, 16))

In [ ]:
# exporti
def to_rgb_block(ni):
    ''' (bs, ni, _, _) -> (bs, 3, _, _) '''
    return nn.Sequential(
        conv2d(ni, 3, 1, 1, 0, bias=False),
        nn.Tanh()
    )

In [ ]:
to_rbg = to_rgb_block(64)
inp = torch.randn(2, 64, 8, 8)
out = to_rbg(inp)
test_eq(out.shape, (2, 3, 8, 8))

In [ ]:
# exporti
def from_rgb_block(nf):
    ''' (bs, 3, _, _) -> (bs, nf, _, _) '''
    return nn.Sequential(
        spectral_conv2d(3, nf, 1, 1, 0, bias=True),
        nn.LeakyReLU(0.2),
    )

In [ ]:
from_rbg = from_rgb_block(64)
inp = torch.randn(2, 3, 8, 8)
out = from_rbg(inp)
test_eq(out.shape, (2, 64, 8, 8))

In [ ]:
# exporti
class SelfAttention(nn.Module):
    ''' expensive operator, usually be used on 64x64 layer, better to not use on layers that larger than 64x64 if you don't have a large GPU
        from https://github.com/ajbrock/BigGAN-PyTorch '''
    def __init__(self, ch):
        super().__init__()
        # Channel multiplier
        self.ch = ch
        self.theta = spectral_conv2d(self.ch, self.ch // 8, 1, 1, 0, bias=False)
        self.phi = spectral_conv2d(self.ch, self.ch // 8, 1, 1, 0, bias=False)
        self.g = spectral_conv2d(self.ch, self.ch // 2, 1, 1, 0, bias=False)
        self.o = spectral_conv2d(self.ch // 2, self.ch, 1, 1, 0, bias=False)
        # Learnable gain parameter
        self.gamma = nn.Parameter(torch.tensor(0.))
    def forward(self, x):
        # Apply convs
        theta = self.theta(x)
        phi = F.max_pool2d(self.phi(x), [2,2])
        g = F.max_pool2d(self.g(x), [2,2])    
        # Perform reshapes
        theta = theta.view(-1, self. ch // 8, x.shape[2] * x.shape[3])
        phi = phi.view(-1, self. ch // 8, x.shape[2] * x.shape[3] // 4)
        g = g.view(-1, self. ch // 2, x.shape[2] * x.shape[3] // 4)
        # Matmul and softmax to get attention maps
        beta = F.softmax(torch.bmm(theta.transpose(1, 2), phi), -1)
        # Attention map times g path
        o = self.o(torch.bmm(g, beta.transpose(1,2)).view(-1, self.ch // 2, x.shape[2], x.shape[3]))
        return self.gamma * o + x

In [ ]:
self_attn = SelfAttention(16)
x = torch.randn(2, 16, 4, 4)
out = self_attn(x)
test_eq(out.shape, (2, 16, 4, 4))

In [ ]:
# exporti
def simple_attn(tgt, src, src_mask=None):
    ''' tgt: (bs, tgt_seq_len, emb_sz), src: (bs, src_seq_len, emb_sz), src_mask: (bs, src_seq_len) [True will be masked]
        returns: (bs, tgt_seq_len, emb_sz), (bs, tgt_seq_len, src_seq_len) '''
    bs, tgt_seq_len, emb_sz = tgt.shape
    _, src_seq_len, _ = src.shape
    
    attn_w = torch.bmm(tgt, src.permute(0, 2, 1)) # (bs, tgt_seq_len, src_seq_len)
#     print(attn_w.view(bs*tgt_seq_len, src_seq_len))
    if src_mask is not None: attn_w.masked_fill_(src_mask.unsqueeze(1), -float('inf')) # (bs, tgt_seq_len, src_seq_len)
    attn_w = torch.softmax(attn_w, 2) # (bs, tgt_seq_len, src_seq_len)
    attn_out = torch.bmm(attn_w, src) # (bs, tgt_seq_len, emb_sz)

    return attn_out, attn_w

In [ ]:
# # exporti
# def simple_attn2(tgt, src, src_mask=None):
#     ''' tgt: (bs, tgt_seq_len, emb_sz), src: (bs, src_seq_len, emb_sz), src_mask: (bs, src_seq_len) [True will be masked]
#         returns: (bs, tgt_seq_len, emb_sz), (bs, tgt_seq_len, src_seq_len) '''
#     bs, tgt_seq_len, emb_sz = tgt.shape
#     _, src_seq_len, _ = src.shape
#     src = src.permute(0, 2, 1) # (bs, emb_sz, src_seq_len)
#     attn_w = torch.bmm(tgt, src) # (bs, tgt_seq_len, src_seq_len)
#     attn_w = attn_w.view(bs*tgt_seq_len, src_seq_len) # (bs*tgt_seq_len, src_seq_len)
    
#     if src_mask is not None:
#         mask = src_mask.repeat(tgt_seq_len, 1) # (bs*tgt_seq_len, src_seq_len)
#         attn_w.masked_fill_(mask, -float('inf'))
    
#     attn_w = nn.functional.softmax(attn_w, dim=1)
#     attn_w = attn_w.view(bs, tgt_seq_len, src_seq_len)
#     attn_w = torch.transpose(attn_w, 1, 2).contiguous() # (bs, src_seq_len, tgt_seq_len)
#     attn_out = torch.bmm(src, attn_w) # (bs, emb_sz, tgt_seq_len)
#     attn_out = attn_out.permute(0, 2, 1) # (bs, tgt_seq_len, emb_sz)
#     attn_w = attn_w.permute(0, 2, 1) # (bs, tgt_seq_len, src_seq_len)

#     return attn_out, attn_w

In [ ]:
tgt = torch.randn(2, 5, 3)
src = torch.randn(2, 4, 3)
src_mask = torch.tensor([[True, False, False, False],
                         [True, True, True, False]])
attn_out, attn_w = simple_attn(tgt, src, src_mask)
test_eq(attn_out.shape, (2, 5, 3))
test_eq(attn_w.shape, (2, 5, 4))

In [ ]:
# exporti
class AttnBlock(nn.Module):
    def __init__(self, emb_sz, nc):
        super().__init__()
        self.emb_sz = emb_sz
        self.nc = nc
        self.conv1 = conv_block_g(emb_sz, nc, 1, 1, 0)
    def forward(self, x, word_emb, src_mask):
        """ Attention from x to word
            x: (bs, nc, h, w), word_emb: (bs, seq_len, emb_sz), src_mask: (bs, seq_len)
            out: (bs, nc, h, w), self.attn_w: (bs, h, w, seq_len) """
        bs, nc, h, w = x.shape
        _, _, emb_sz = word_emb.shape
        assert (nc, emb_sz) == (self.nc, self.emb_sz)
        tgt = x.view(bs, nc, -1).permute(0, 2, 1).contiguous() # (bs, h*w, nc)
        
        # Use Conv1x1 to reduce word_emb dim, Do Not Use Linear!!!
        word_emb_t = word_emb.permute(0, 2, 1).unsqueeze(3).contiguous() # (bs, emb_sz, seq_len, 1)
        src = self.conv1(word_emb_t).squeeze(3) # (bs, nc, seq_len)
        src = src.permute(0, 2, 1).contiguous() # (bs, seq_len, nc)
        
        attn_out, attn_w = simple_attn(tgt, src, src_mask=src_mask) # (bs, h*w, nc), (bs, h*w, seq_len)
        attn_out = attn_out.permute(0, 2, 1).view(bs, nc, h, w) # (bs, nc, h, w)
        self.attn_w = attn_w.view(bs, h, w, -1) # (bs, h, w, seq_len)
        
        out = x + attn_out # (bs, nc, h, w)
#         out = torch.cat([x, attn_out], dim=1) # (bs, 2*nc, h, w)
        return out

In [ ]:
attn_block = AttnBlock(600, 256)
x = torch.randn(16, 256, 16, 16)
word_emb = torch.randn(16, 20, 600)
src_mask = torch.ones(16, 20).bool()
out = attn_block(x, word_emb, src_mask)
test_eq(out.shape, (16, 256, 16, 16))
test_eq(attn_block.attn_w.shape, (16, 16, 16, 20))

## Generator

In [ ]:
# exporti

# CH_TABLE = [512, 512, 512, 512, 256, 128,  64,  32,  16]
#             4      8   16   32   64  128  256  512 1024

CH_TABLE = [1024, 512, 256, 128,  64,  64,  64,  64,  64]
#              4    8   16   32   64  128  256  512 1024

# CH_TABLE = [512, 384, 256, 192, 128,  96,  64,  64,  64]
#             4    8   16   32   64  128  256  512 1024

In [ ]:
# exporti
class G_Init(nn.Module):
    def __init__(self, emb_sz, noise_sz=100):
        super().__init__()
        self.noise_sz = noise_sz
        self.conv = nn.Sequential(
            conv_block_g(emb_sz+noise_sz, CH_TABLE[0], 4, 1, 3), # (bs, CH_TABLE[0], 4, 4)
            ConvResBlockG(CH_TABLE[0]), # (bs, CH_TABLE[0], 4, 4)
        )
        self.to_rgb = to_rgb_block(CH_TABLE[0]) # (bs, 3, 4, 4)
    def forward(self, sent_emb, noise):
        ''' sent_emb: (bs, emb_sz), noise: (bs, noise_sz)
            returns: img(bs, 3, 4, 4), code(bs, 512, 4, 4) '''
        assert noise.shape[1]==self.noise_sz
        code = torch.cat([sent_emb, noise], 1)[..., None, None] # (bs, emb_sz+noise_sz, 1, 1)
        code = self.conv(code) # (bs, CH_TABLE[0], 4, 4)
        img = self.to_rgb(code) # (bs, 3, 4, 4)
        return img, code

In [ ]:
g_init = G_Init(25, 100)
sent_emb = torch.randn(2, 25)
noise = torch.randn(2, 100)
img, code = g_init(sent_emb, noise)
test_eq(img.shape, (2, 3, 4, 4))
test_eq(code.shape, (2, CH_TABLE[0], 4, 4))

In [ ]:
# exporti
class G_General(nn.Module):
    def __init__(self, ni, nf, is_self_attn=False):
        super().__init__()
        self.conv = nn.Sequential(
            up_block(),
            conv_block_g(ni, nf), # (bs, nf, inp_sz*2, inp_sz*2)
            ConvResBlockG(nf), # (bs, nf, inp_sz*2, inp_sz*2)
            SelfAttention(nf) if is_self_attn else IdentityModule(), # (bs, nf, inp_sz*2, inp_sz*2)
        )
        self.to_rgb = to_rgb_block(nf) # (bs, nf, inp_sz*2, inp_sz*2)
    def forward(self, code):
        ''' code: (bs, ni, inp_sz, inp_sz)
            returns: img(bs, 3, inp_sz*2, inp_sz*2), code(bs, nf, inp_sz*2, inp_sz*2) '''
        code = self.conv(code) # (bs, nf, inp_sz*2, inp_sz*2)
        img = self.to_rgb(code) # (bs, 3, inp_sz*2, inp_sz*2)
        return img, code

In [ ]:
g_general = G_General(512, 256, is_self_attn=True)
code = torch.randn(2, 512, 4, 4)
img, code = g_general(code)
test_eq(img.shape, (2, 3, 8, 8))
test_eq(code.shape, (2, 256, 8, 8))

In [ ]:
# exporti
class G_General_Attn(nn.Module):
    def __init__(self, emb_sz, ni, nf, is_self_attn=False):
        super().__init__()
        self.conv = MultiSequential(
            AttnBlock(emb_sz, ni), # (bs, ni, inp_sz, inp_sz)
            conv_block_g(ni, nf), # (bs, nf, inp_sz, inp_sz)
            up_block(), # (bs, nf, inp_sz*2, inp_sz*2)
            ConvResBlockG(nf), # (bs, nf, inp_sz*2, inp_sz*2)
            SelfAttention(nf) if is_self_attn else IdentityModule(), # (bs, nf, inp_sz*2, inp_sz*2)
        )
        self.to_rgb = to_rgb_block(nf) # (bs, 3, inp_sz*2, inp_sz*2)
    def forward(self, code, word_emb, src_mask):
        ''' code: (bs, ni, inp_sz, inp_sz), word_emb: (bs, seq_len, emb_sz), src_mask: (bs, seq_len) 
            returns: img(bs, 3, inp_sz*2, inp_sz*2), code(bs, nf, inp_sz*2, inp_sz*2) '''
        code = self.conv(code, word_emb, src_mask) # (bs, nf, inp_sz*2, inp_sz*2)
        img = self.to_rgb(code) # (bs, 3, inp_sz*2, inp_sz*2)
        return img, code

In [ ]:
g_general_attn = G_General_Attn(25, 512, 256, is_self_attn=True)
code = torch.randn(2, 512, 4, 4)
word_emb = torch.randn(2, 2, 25)
src_mask = torch.ones(2, 2).bool()
img, code = g_general_attn(code, word_emb, src_mask)
test_eq(img.shape, (2, 3, 8, 8))
test_eq(code.shape, (2, 256, 8, 8))

In [ ]:
# export
class Anime_G(nn.Module):
    def __init__(self, emb_sz, noise_sz=100):
        super().__init__()
        self.g_init = G_Init(emb_sz, noise_sz) # (bs, 3, 4, 4), (bs, CH_TABLE[0], 4, 4)
        self.gs = nn.ModuleList([
            G_General(CH_TABLE[0], CH_TABLE[1]), # (bs, 3, 8, 8), (bs, CH_TABLE[1], 8, 8)
            G_General(CH_TABLE[1], CH_TABLE[2]), # (bs, 3, 16, 16), (bs, CH_TABLE[2], 16, 16)
        ])
        self.gs_attn = nn.ModuleList([
            G_General_Attn(emb_sz, CH_TABLE[2], CH_TABLE[3]), # (bs, 3, 32, 32), (bs, CH_TABLE[3], 32, 32)
            G_General_Attn(emb_sz, CH_TABLE[3], CH_TABLE[4], is_self_attn=True), # (bs, 3, 64, 64), (bs, CH_TABLE[4], 64, 64)
        ])
    def forward(self, sent_emb, noise, word_emb, src_mask):
        ''' sent_emb: (bs, emb_sz), noise: (bs, noise_sz), word_emb: (bs, seq_len, emb_sz), src_mask: (bs, seq_len) 
            returns: imgs: [(bs, 3, 4, 4),...,(bs, 3, 64, 64)] '''
        imgs = []
        img, code = self.g_init(sent_emb, noise) # (bs, 3, 4, 4), (bs, CH_TABLE[0], 4, 4)
        imgs.append(img)
        for g in self.gs:
            img, code = g(code)
            imgs.append(img)
        for g in self.gs_attn:
            img, code = g(code, word_emb, src_mask)
            imgs.append(img)
        return imgs

In [ ]:
anime_g = Anime_G(25, 100)
sent_emb = torch.randn(2, 25)
noise = torch.randn(2, 100)
word_emb = torch.randn(2, 2, 25)
src_mask = torch.ones(2, 2).bool()
imgs = anime_g(sent_emb, noise, word_emb, src_mask)
test_eq([img.shape for img in imgs], 
        [torch.Size([2, 3, 4, 4]), 
         torch.Size([2, 3, 8, 8]), 
         torch.Size([2, 3, 16, 16]), 
         torch.Size([2, 3, 32, 32]), 
         torch.Size([2, 3, 64, 64])])

In [ ]:
# export
class Birds_G(nn.Module):
    def __init__(self, emb_sz, noise_sz=100):
        super().__init__()
        self.g_init = G_Init(emb_sz, noise_sz) # (bs, 3, 4, 4), (bs, CH_TABLE[0], 4, 4)
        self.gs = nn.ModuleList([
            G_General(CH_TABLE[0], CH_TABLE[1]), # (bs, 3, 8, 8), (bs, CH_TABLE[1], 8, 8)
            G_General(CH_TABLE[1], CH_TABLE[2]), # (bs, 3, 16, 16), (bs, CH_TABLE[2], 16, 16)
            G_General(CH_TABLE[2], CH_TABLE[3]), # (bs, 3, 32, 32), (bs, CH_TABLE[3], 32, 32)
            G_General(CH_TABLE[3], CH_TABLE[4], is_self_attn=True), # (bs, 3, 64, 64), (bs, CH_TABLE[4], 64, 64)
        ])
        self.gs_attn = nn.ModuleList([
            G_General_Attn(emb_sz, CH_TABLE[4], CH_TABLE[5]), # (bs, 3, 128, 128), (bs, CH_TABLE[5], 128, 128)
            G_General_Attn(emb_sz, CH_TABLE[5], CH_TABLE[6]), # (bs, 3, 256, 256), (bs, CH_TABLE[6], 256, 256)
        ])
    def forward(self, sent_emb, noise, word_emb, src_mask):
        ''' sent_emb: (bs, emb_sz), noise: (bs, noise_sz), word_emb: (bs, seq_len, emb_sz), src_mask: (bs, seq_len) 
            returns: imgs: [(bs, 3, 4, 4),...,(bs, 3, 256, 256)], code: (bs, 64, 256, 256)'''
        imgs = []
        img, code = self.g_init(sent_emb, noise) # (bs, 3, 4, 4), (bs, 512, 4, 4)
        imgs.append(img)
        for g in self.gs:
            img, code = g(code)
            imgs.append(img)
        for g in self.gs_attn:
            img, code = g(code, word_emb, src_mask)
            imgs.append(img)
        return imgs

In [ ]:
birds_g = Birds_G(25, 100)
sent_emb = torch.randn(2, 25)
noise = torch.randn(2, 100)
word_emb = torch.randn(2, 2, 25)
src_mask = torch.ones(2, 2).bool()
imgs = birds_g(sent_emb, noise, word_emb, src_mask)
test_eq([img.shape for img in imgs], 
        [torch.Size([2, 3, 4, 4]), 
         torch.Size([2, 3, 8, 8]), 
         torch.Size([2, 3, 16, 16]), 
         torch.Size([2, 3, 32, 32]), 
         torch.Size([2, 3, 64, 64]),
         torch.Size([2, 3, 128, 128]),
         torch.Size([2, 3, 256, 256])])

## Discriminator

In [ ]:
# exporti
class UncondCls(nn.Module):
    def __init__(self):
        super().__init__()
        self.cls = nn.Sequential(
            conv2d(CH_TABLE[0]+1, 1, 4, 1, 0, bias=False), # (bs, 1, 1, 1)
            nn.Flatten(), # (bs, 1)
        )
    def forward(self, sent_code):
        ''' sent_code: (bs, CH_TABLE[0], 4, 4), returns: (bs, 1) '''
        return self.cls(sent_code)

In [ ]:
sent_code = torch.randn(2, CH_TABLE[0]+1, 4, 4)
uncond_cls = UncondCls()
uncond_logit = uncond_cls(sent_code)
test_eq(uncond_logit.shape, (2, 1))

In [ ]:
# exporti
class CondCls(nn.Module):
    def __init__(self, emb_sz):
        super().__init__()
        self.up = nn.Upsample(4)
        self.cls = nn.Sequential(
            conv_block_d(CH_TABLE[0]+1+emb_sz, CH_TABLE[0]), # (bs, CH_TABLE[0], 4, 4)
            conv2d(CH_TABLE[0], 1, 4, 1, 0, bias=False),
            nn.Flatten(), # (bs, 1)
        )
    def forward(self, sent_code, sent_emb):
        ''' sent_code: (bs, CH_TABLE[0], 4, 4), sent_emb: (bs, emb_sz)
            returns: (bs, 1) '''
        sent_emb = self.up(sent_emb[..., None, None]) # (bs, emb_sz, 4, 4)
        sent_code = torch.cat([sent_code, sent_emb], 1) # (bs, emb_sz+CH_TABLE[0], 4, 4)
        return self.cls(sent_code)

In [ ]:
sent_code = torch.randn(2, CH_TABLE[0]+1, 4, 4)
sent_emb = torch.randn(2, 25) 
cond_cls = CondCls(25)
cond_logit = cond_cls(sent_code, sent_emb)
test_eq(cond_logit.shape, (2, 1))

In [ ]:
CH_TABLE

[1024, 512, 256, 128, 64, 64, 64, 64, 64]

In [ ]:
# exporti
class D_General(nn.Module):
    def __init__(self, ni, nf, is_first=False, is_self_attn=False):
        super().__init__()
        self.is_first = is_first
        self.from_rgb = from_rgb_block(ni) # (bs, ni, inp_sz, inp_sz)
        self.conv = nn.Sequential(
            SelfAttention(ni) if is_self_attn else IdentityModule(), # (bs, ni, inp_sz, inp_sz)
            ConvResBlockD(ni), # (bs, 2*ni, inp_sz, inp_sz)
            conv_block_d(ni, nf), # (bs, nf, inp_sz, inp_sz)
            down_block(), # (bs, nf, inp_sz//2, inp_sz//2)
        )
    def forward(self, img, code=None):
        """ img: (bs, 3, inp_sz, inp_sz), code: (bs, ni, inp_sz, inp_sz)
            returns: (bs, nf, inp_sz//2, inp_sz//2) """
        assert (code is None and self.is_first==True) or (code is not None and self.is_first==False)
        code2 = self.from_rgb(img) # (bs, ni, inp_sz, inp_sz)
        if self.is_first:
            code = code2
        else:
            code = code + code2
        code = self.conv(code) # (bs, nf, inp_sz//2, inp_sz//2)
        return code

In [ ]:
d_general = D_General(16, 32, is_self_attn=True)
img = torch.randn(2, 3, 8, 8)
code = torch.randn(2, 16, 8, 8)
code = d_general(img, code)
test_eq(code.shape, (2, 32, 4, 4))

In [ ]:
# export
class Anime_D(nn.Module):
    def __init__(self, emb_sz):
        super().__init__()
        self.ds = nn.ModuleList([
            D_General(CH_TABLE[4], CH_TABLE[3], is_first=True, is_self_attn=True), # (bs, CH_TABLE[3], 32, 32)
            D_General(CH_TABLE[3], CH_TABLE[2]), # (bs, CH_TABLE[2], 16, 16)
            D_General(CH_TABLE[2], CH_TABLE[1]), # (bs, CH_TABLE[1], 8, 8)
            D_General(CH_TABLE[1], CH_TABLE[0]), # (bs, CH_TABLE[0], 4, 4)
        ])
        self.minibatch_std_dev = MinibatchStdDev()
        self.from_rgb = from_rgb_block(CH_TABLE[0])
        self.uncond_cls = UncondCls()
        self.cond_cls = CondCls(emb_sz)
    def get_sent_code(self, imgs):
        ''' imgs: [(2, 3, 64, 64), ..., (bs, 3, 4, 4)]
            returns: (bs, CH_TABLE[0]+1, 4, 4) '''
        assert len(imgs)==5
        code = None
        for i in range(len(self.ds)):
            code = self.ds[i](imgs[i], code) # (bs, CH_TABLE[0], 4, 4)
        code2 = self.from_rgb(imgs[-1]) # (bs, CH_TABLE[0], 4, 4)
        sent_code = self.minibatch_std_dev(code + code2)
        return sent_code
    def forward(self, imgs, sent_emb):
        ''' imgs: [(2, 3, 64, 64), ..., (bs, 3, 4, 4)], sent_emb: (bs, emb_sz)
            returns: uncond_logit(bs, 1), cond_logit(bs, 1) '''
        sent_code = self.get_sent_code(imgs) # (bs, CH_TABLE[0], 4, 4)
        uncond_logit = self.uncond_cls(sent_code)
        cond_logit = self.cond_cls(sent_code, sent_emb)
        return uncond_logit, cond_logit

In [ ]:
anime_d = Anime_D(25)
imgs =[torch.randn(2, 3, 64, 64), torch.randn(2, 3, 32, 32), torch.randn(2, 3, 16, 16), torch.randn(2, 3, 8, 8), torch.randn(2, 3, 4, 4)]
sent_emb = torch.randn(2, 25)

sent_code = anime_d.get_sent_code(imgs)
test_eq(sent_code.shape, (2, CH_TABLE[0]+1, 4, 4))

uncond_logit, cond_logit = anime_d(imgs, sent_emb)
test_eq(uncond_logit.shape, (2, 1))
test_eq(cond_logit.shape, (2, 1))

In [ ]:
# export
class Birds_D(nn.Module):
    def __init__(self, emb_sz):
        super().__init__()
        self.ds = nn.ModuleList([
            D_General(CH_TABLE[6], CH_TABLE[5], is_first=True), # (bs, CH_TABLE[5], 128, 128)
            D_General(CH_TABLE[5], CH_TABLE[4]), # (bs, CH_TABLE[4], 64, 64)
            D_General(CH_TABLE[4], CH_TABLE[3], is_self_attn=True), # (bs, CH_TABLE[3], 32, 32)
            D_General(CH_TABLE[3], CH_TABLE[2]), # (bs, CH_TABLE[2], 16, 16)
            D_General(CH_TABLE[2], CH_TABLE[1]), # (bs, CH_TABLE[1], 8, 8)
            D_General(CH_TABLE[1], CH_TABLE[0]), # (bs, CH_TABLE[0], 4, 4)
        ])
        self.minibatch_std_dev = MinibatchStdDev()
        self.from_rgb = from_rgb_block(CH_TABLE[0])
        self.uncond_cls = UncondCls()
        self.cond_cls = CondCls(emb_sz)
    def get_sent_code(self, imgs):
        ''' imgs: [(2, 3, 256, 256), ..., (bs, 3, 4, 4)]
            returns: (bs, CH_TABLE[0]+1, 4, 4) '''
        assert len(imgs)==7
        code = None
        for i in range(len(self.ds)):
            code = self.ds[i](imgs[i], code) # (bs, CH_TABLE[0], 4, 4)
        code2 = self.from_rgb(imgs[-1]) # (bs, CH_TABLE[0], 4, 4)
        sent_code = self.minibatch_std_dev(code + code2)
        return sent_code
    def forward(self, imgs, sent_emb):
        ''' imgs: [(2, 3, 256, 256), ..., (bs, 3, 4, 4)], sent_emb: (bs, emb_sz)
            returns: uncond_logit(bs, 1), cond_logit(bs, 1) '''
        sent_code = self.get_sent_code(imgs)
        uncond_logit = self.uncond_cls(sent_code)
        cond_logit = self.cond_cls(sent_code, sent_emb)
        return uncond_logit, cond_logit

In [ ]:
birds_d = Birds_D(25)
imgs =[torch.randn(2, 3, 256, 256), torch.randn(2, 3, 128, 128), torch.randn(2, 3, 64, 64), torch.randn(2, 3, 32, 32), torch.randn(2, 3, 16, 16), torch.randn(2, 3, 8, 8), torch.randn(2, 3, 4, 4)]
sent_emb = torch.randn(2, 25)

sent_code = birds_d.get_sent_code(imgs)
test_eq(sent_code.shape, (2, CH_TABLE[0]+1, 4, 4))

uncond_logit, cond_logit = birds_d(imgs, sent_emb)
test_eq(uncond_logit.shape, (2, 1))
test_eq(cond_logit.shape, (2, 1))

## Exported Model

In [ ]:
# export
class Anime_Export(nn.Module):
    def __init__(self, vocab_sz, emb_sz, pad_id, rnn_layers=2, rnn_drop_p=0.5, noise_sz=100):
        super().__init__()
        self.noise_sz = noise_sz
        self.rnn_encoder = RnnEncoder(vocab_sz, emb_sz, pad_id, n_layers=rnn_layers, drop_p=rnn_drop_p).requires_grad_(False).eval()
        self.g_net = Anime_G(emb_sz, noise_sz)
#         self.samples = ( torch.tensor([[1, 2],[3, 4]]), torch.tensor([2, 2]) ) # (input_ids, cap_len)
    def forward(self, inp_ids, cap_len):
        """ inp_ids: (bs, max_seq_len), cap_len: (bs,)  
            returns: (bs, 3, 64, 64) """
        bs, max_seq_len = inp_ids.shape
        device = inp_ids.device
        src_mask = get_src_mask(cap_len, max_seq_len, device)
        sent_emb, word_emb = self.rnn_encoder(inp_ids, cap_len)
        noise = noise_gen.sample((bs, self.noise_sz)).to(device)
        imgs = self.g_net(sent_emb, noise, word_emb, src_mask)
        return imgs[-1]
    @classmethod
    def from_pretrained(cls, path, device='cpu'):
        state = torch.load(path, map_location=device)
        vocab_sz = state['vocab_sz']
        emb_sz = state['emb_sz']
        pad_id = state['pad_id']
        rnn_layers = state['rnn_layers']
        rnn_drop_p = state['rnn_drop_p']
        noise_sz = state['noise_sz']
        m = cls(vocab_sz, emb_sz, pad_id, rnn_layers, rnn_drop_p, noise_sz)
        m.rnn_encoder.load_state_dict(state['rnn_encoder'])
        m.g_net.load_state_dict(state['g_net'])
#         m.samples = state['samples']
        return m
#     def small_forward(self, inp_ids, cap_len):
#         """ inp_ids: (bs, seq_len) for bs is small, cap_len: (bs,)
#             returns: (bs+self.samples.shape[0], 3, 64, 64) """
#         inp_ids = torch.cat([inp_ids, self.samples[0]])
#         cap_len = torch.cat([cap_len, self.samples[1]])
#         return self(inp_ids, cap_len)

In [ ]:
anime_export = Anime_Export(5, 24, 0)
inp_ids = torch.tensor([[1, 2],
                        [3, 4]])
cap_len = torch.tensor([2, 2])
img = anime_export(inp_ids, cap_len)
test_eq(img.shape, (2, 3, 64, 64))

In [ ]:
# anime_export = Anime_Export(5, 24, 0)
# inp_ids = torch.tensor([[1, 2],
#                         [3, 4]])
# cap_len = torch.tensor([2, 2])
# img = anime_export.small_forward(inp_ids, cap_len)
# test_eq(img.shape, (anime_export.samples[0].shape[0]+2, 3, 64, 64))

In [ ]:
# export
class Birds_Export(nn.Module):
    def __init__(self, vocab_sz, emb_sz, pad_id, rnn_layers=2, rnn_drop_p=0.5, noise_sz=100):
        super().__init__()
        self.noise_sz = noise_sz
        self.rnn_encoder = RnnEncoder(vocab_sz, emb_sz, pad_id, n_layers=rnn_layers, drop_p=rnn_drop_p).requires_grad_(False).eval()
        self.g_net = Birds_G(emb_sz, noise_sz)
#         self.samples = ( torch.randint(0, 4, (2, 25)), torch.tensor( [8, 4]) ) # (input_ids, cap_len)
    def forward(self, inp_ids, cap_len):
        """ inp_ids: (bs, max_seq_len), cap_len: (bs,)  
            returns: (bs, 3, 256, 256) """
        bs, max_seq_len = inp_ids.shape
        device = inp_ids.device
        src_mask = get_src_mask(cap_len, max_seq_len, device)
        sent_emb, word_emb = self.rnn_encoder(inp_ids, cap_len)
        noise = noise_gen.sample((bs, self.noise_sz)).to(device)
        imgs = self.g_net(sent_emb, noise, word_emb, src_mask)
        return imgs[-1]
    @classmethod
    def from_pretrained(cls, path, device='cpu'):
        state = torch.load(path, map_location=device)
        vocab_sz = state['vocab_sz']
        emb_sz = state['emb_sz']
        pad_id = state['pad_id']
        rnn_layers = state['rnn_layers']
        rnn_drop_p = state['rnn_drop_p']
        noise_sz = state['noise_sz']
        m = cls(vocab_sz, emb_sz, pad_id, rnn_layers, rnn_drop_p, noise_sz)
        m.rnn_encoder.load_state_dict(state['rnn_encoder'])
        m.g_net.load_state_dict(state['g_net'])
#         m.samples = state['samples']
        return m
#     def small_forward(self, inp_ids, cap_len):
#         """ inp_ids: (bs, seq_len) for bs is small, cap_len: (bs,)
#             returns: (bs+self.samples.shape[0], 3, 256, 256) """
#         inp_ids = torch.cat([inp_ids, self.samples[0]])
#         cap_len = torch.cat([cap_len, self.samples[1]])
#         return self(inp_ids, cap_len)

In [ ]:
birds_export = Birds_Export(5, 24, 0)
inp_ids = torch.randint(0, 4, (2, 25))
cap_len = torch.tensor([2, 2])
img = birds_export(inp_ids, cap_len)
test_eq(img.shape, (2, 3, 256, 256))

In [ ]:
# birds_export = Birds_Export(5, 24, 0)
# inp_ids = torch.randint(0, 4, (2, 25))
# cap_len = torch.tensor([2, 2])
# img = birds_export.small_forward(inp_ids, cap_len)
# test_eq(img.shape, (anime_export.samples[0].shape[0]+2, 3, 256, 256))

## Export -

In [3]:
# hide
from nbdev.export import notebook2script
notebook2script()

Converted 00a_torch_utils.ipynb.
Converted 02a_data_anime_heads.ipynb.
Converted 02b_data_birds.ipynb.
Converted 03a_model.ipynb.
Converted 04a_trainer_DAMSM.ipynb.
Converted 04b_trainer_GAN-Copy1.ipynb.
Converted 05a_inference_anime_heads.ipynb.
Converted 05b_inference_birds.ipynb.
Converted 99a_Train_Anime_Heads_DAMSM.ipynb.
Converted 99b_Train_Anime_Heads_GAN.ipynb.
Converted 99c_Train_Birds_DAMSM.ipynb.
Converted 99d_Train_Birds_GAN.ipynb.
Converted Untitled.ipynb.
Converted index.ipynb.
